In [1]:
import numpy as np
import os
import glob
import gzip
import networkx as nx
import pandas as pd
from decimal import Decimal, getcontext
from time import process_time

In [6]:
N = 100000
dim = 1
alpha_a = 1.0
alpha_g = 1.0
#path_d = f"../../../data/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/gml"
path_d = f"../../../data/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/gml/"
#conditional_ = os.path.exists(path_d)
#if(conditional_ == True):
    #print("exist")
#else:
    #print("not exist")
#all_files = glob.glob(os.path.join(path_d,"*.gz.gml"))
all_files = glob.glob(os.path.join(path_d,"*.gz"))
for file in all_files:
    print(os.path.basename(file)[4:-7])
#print(all_files)

53
1018130995
1041164526
1091239781
1093996181
1183285891
1205053376
1325199855
1338885646
1398167334
1430498129
1437533619
1437717624
1442988960
1518754475
1573396877
1573470506
1585335616
1600706525
1670399748
167499480
1702780342
1748791481
184755188
2008091988
2036436768
2076995969
2083832329
2085597812
2092581796
2108082694
211572122
2131905456
2133962605
436980026
44694364
476519248
482093731
495979758
574683016
640540855
668978314
711277376
718310521
725693024
75555091
763159336
783292180
808294238
8840061
886123487
935357653
988500446
990223432


In [6]:
N = 100000
dim = 1
alpha_a = 1.0
alpha_g = 2.0

if(alpha_g==0.0):
    pass
else:
    # Directory with all samples
    path_d = f"../../../data/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/gml/"
    # dataframe with all samples
    new_file = "/properties_set_r.txt"

    # Check if directory exist
    conditional_ = os.path.exists(path_d)
    if(conditional_ == True):
        pass
    else:
        print("data doesn't exist, run code in c++ to gen data")

    # Check if file exist
    check_file = os.path.isfile(path_d+new_file)

    # Open all files path in directory .csv
    all_files = glob.glob(os.path.join(path_d,"*.gz"))
    # If file exist, open
    if(check_file == True):
        df = pd.read_csv(path_d+new_file,sep=" ")
        #filter_list to check if files are in dataframe
        filter_list = str(df["#cod_file"].values) 
        num_samples = len(df)
        for file in all_files:
            # Check if file are in dataframe
            conditional = os.path.basename(file)[4:-7] in filter_list
            # Make nothing if True conditional
            if(conditional==True):
                pass
            # Add new elements in dataframe
            else:
                # load node properties
                node = {"id": [],
                "position":[],
                "degree": []}
                
                # load edge properties
                edge = {"connections": [],
                        "distance": []}
                
                with gzip.open(file) as file_in:
                    String = file_in.readlines()
                    Lines = [i.decode('utf-8') for i in String]
                    for i in range(len(Lines)):
                        if(Lines[i]=='node\n'):
                            node["id"].append(int(Lines[i+2][4:-2]))
                            node["position"].append([float(Lines[i+6][2:-1]),float(Lines[i+7][2:-1]),float(Lines[i+8][2:-1])])
                            if(Lines[i+9][0]=='q'):
                                node["degree"].append(int(Lines[i+10][7:-1]))
                            else:
                                node["degree"].append(int(Lines[i+9][7:-1]))
                        elif(Lines[i]=="edge\n"):
                            edge["connections"].append([int(Lines[i+2][8:-2]),int(Lines[i+3][8:-2])])
                            edge["distance"].append(float(Lines[i+4][9:-1]))
                
                D = np.array(node["degree"])
                getcontext().prec = 50  # Set precision to 50 decimal places
                Ter_1 = Decimal(int(sum(D)))
                Ter_3 = Decimal(int(np.dot(D,D)))
                Ter_4 = Decimal(int(sum(D**3)))
                
                G = nx.from_edgelist(edge["connections"])
                Ter_2 = 0
                
                for j in G.edges():
                    d_s = G.degree[j[0]]
                    d_t = G.degree[j[1]]
                    Ter_2 += d_s*d_t 
                
                Ter_2 = Decimal(Ter_2)
                
                getcontext().prec = 10  # Set precision to 50 decimal places
                
                r = Decimal((Ter_1*Ter_2-Ter_3*2)/(Ter_1*Ter_4-Ter_3**2))
                df.loc[num_samples,"#ass_coeff"] = r
                df.loc[num_samples,"#cod_file"] = os.path.basename(file)[4:-7]
                num_samples += 1
        # Save new dataframe update
        df["#cod_file"] = df["#cod_file"].astype(int)
        df.to_csv(path_d+new_file,sep=' ',index=False)

    # Else, create it
    else:
        ass_coeff = []
        cod_file = []
        # Open all files path in directory .csv
        
        for file in all_files:
            node = {"id": [],
                "position":[],
                "degree": []}
            edge = {"connections": [],
                    "distance": []}
            with gzip.open(file) as file_in:
                String = file_in.readlines()
                Lines = [i.decode('utf-8') for i in String]
                for i in range(len(Lines)):
                    if(Lines[i]=='node\n'):
                        node["id"].append(int(Lines[i+2][4:-2]))
                        node["position"].append([float(Lines[i+6][2:-1]),float(Lines[i+7][2:-1]),float(Lines[i+8][2:-1])])
                        if(Lines[i+9][0]=='q'):
                            node["degree"].append(int(Lines[i+10][7:-1]))
                        else:
                            node["degree"].append(int(Lines[i+9][7:-1]))
                    elif(Lines[i]=="edge\n"):
                        edge["connections"].append([int(Lines[i+2][8:-2]),int(Lines[i+3][8:-2])])
                        edge["distance"].append(float(Lines[i+4][9:-1]))
            D = np.array(node["degree"])
            print(D)
            getcontext().prec = 50  # Set precision to 50 decimal places
            Ter_1 = Decimal(int(sum(D)))
            Ter_3 = Decimal(int(np.dot(D,D)))
            Ter_4 = Decimal(int(sum(D**3)))
            G = nx.from_edgelist(edge["connections"])
            Ter_2 = 0
            for j in G.edges():
                d_s = G.degree[j[0]]
                d_t = G.degree[j[1]]
                Ter_2 += d_s*d_t 
            Ter_2 = Decimal(Ter_2)
            getcontext().prec = 10  # Set precision to 50 decimal places
            r = Decimal((Ter_1*Ter_2-Ter_3*2)/(Ter_1*Ter_4-Ter_3**2))
            
            ass_coeff.append(r)
            cod_file.append(os.path.basename(file)[4:-7])
        df = pd.DataFrame(data={"#ass_coeff":ass_coeff,"#cod_file":cod_file})
        df.to_csv(path_d+new_file,sep=' ',index=False)

[ 32  27 159 ...   1   1   1]
[19  9 14 ...  1  1  1]
[110  12  15 ...   1   1   1]
[ 41 150  31 ...   1   1   1]
[20 23 73 ...  1  1  1]
[ 7 46 19 ...  1  1  1]
[  4 434 120 ...   1   1   1]
[61 68 13 ...  1  1  1]
[144  27  11 ...   1   1   1]
[110  14  25 ...   1   1   1]
[205  53  21 ...   1   1   1]
[160  44  52 ...   1   1   1]
[ 26 231  11 ...   1   1   1]
[ 22 158  13 ...   1   1   1]
[87 14  1 ...  1  1  1]
[ 9 55 83 ...  1  1  1]
[  8  87 104 ...   1   1   1]
[22 29 39 ...  1  1  1]
[ 79 140  15 ...   1   1   1]
[ 8 67 16 ...  1  1  1]
[ 27 100 155 ...   1   1   1]
[242  71  16 ...   1   1   1]
[ 85 155  14 ...   1   1   1]
[17 68  8 ...  1  1  1]
[ 29 230   2 ...   1   1   1]
[ 11  41 211 ...   1   1   1]
[ 17  15 240 ...   1   1   1]
[14 87 65 ...  1  1  1]
[19  8 48 ...  1  1  1]
[53 26  4 ...  1  1  1]
[181  54  27 ...   1   1   1]
[ 84 109  17 ...   1   1   1]
[ 13 224  17 ...   1   1   1]
[13 87 49 ...  1  1  1]
[ 33  13 134 ...   1   1   1]
[273   4  94 ...   1   1   1